Source: [Speed Up Your Python Program with Concurrency](http://www.pybloggers.com/2019/01/speed-up-your-python-program-with-concurrency/)

In [1]:
import requests
import time

def download_site(url, session):
    with session.get(url) as response:
        print(f"Read {len(response.content)} from {url}")
        
def download_all_sites(sites):
    with requests.session() as session:
        for url in sites:
            download_site(url, session)
            
if __name__ == "__main__":
    sites = [
        "http://www.jython.org",
        "http://olympus.realpython.org/dice",
    ] * 80

start_time = time.time()
download_all_sites(sites)
duration = time.time() - start_time
print(f"Downloaded {len(sites)} in {duration} seconds")

Read 19210 from http://www.jython.org
Read 276 from http://olympus.realpython.org/dice
Read 19210 from http://www.jython.org
Read 276 from http://olympus.realpython.org/dice
Read 19210 from http://www.jython.org
Read 276 from http://olympus.realpython.org/dice
Read 19210 from http://www.jython.org
Read 276 from http://olympus.realpython.org/dice
Read 19210 from http://www.jython.org
Read 276 from http://olympus.realpython.org/dice
Read 19210 from http://www.jython.org
Read 276 from http://olympus.realpython.org/dice
Read 19210 from http://www.jython.org
Read 276 from http://olympus.realpython.org/dice
Read 19210 from http://www.jython.org
Read 276 from http://olympus.realpython.org/dice
Read 19210 from http://www.jython.org
Read 276 from http://olympus.realpython.org/dice
Read 19210 from http://www.jython.org
Read 276 from http://olympus.realpython.org/dice
Read 19210 from http://www.jython.org
Read 276 from http://olympus.realpython.org/dice
Read 19210 from http://www.jython.org
Read 

In [5]:
import concurrent.futures
import requests
import threading
import time

thread_local =  threading.local()

def get_session():
    if not getattr(thread_loacal, "session", None):
        thread.local.session = requests.Session()
    return thread_local.session

def download_site(url):
    session = get_session()
    with session.get(url) as response:
        print(f"Read {len(response.content)} from {url}")
        
def download_all_sites(sites):
    with concurrent.futures.ThreadPoolExecutor(max_workers =5) as executor:
        executor.map(download_site, sites)
        
if __name__ == "__main__":
     sites = [
        "http://www.jython.org",
        "http://olympus.realpython.org/dice",
    ] * 80

start_time = time.time()
download_all_sites(sites)
duration = time.time() - start_time
print(f"Downloaded {len(sites)} in {duration} seconds")

Downloaded 160 in 0.04680013656616211 seconds


### `asyncio` Version ###

The below code is running when executed as a script in command prompt.

In [2]:
import asyncio
import time
import aiohttp

async def download_site(session, url):
    async with session.get(url) as response:
        print("Read {0} from {1}".format(response.content_length, url))
        
async def download_all_sites(sites):
    async with aiohttp.ClientSession() as session:
        tasks = []
        for  url in sites:
            task = asyncio.ensure_future(download_site(session, url))
            tasks.append(task)
        await asyncio.gather(*tasks,return_exceptions = True)
        
if __name__ == "__main__":
    sites = ["http://www.jython.org","http://olympus.realpython.org/dice"] * 80
    start_time = time.time()
    asyncio.get_event_loop().run_until_complete(download_all_sites(sites))
    duration = time.time() - start_time
    print(f"Downloaded {len(sites)} in {duration} seconds")        

RuntimeError: This event loop is already running

Read 19210 from http://www.jython.org
Read 19210 from http://www.jython.org
Read 19210 from http://www.jython.org
Read 19210 from http://www.jython.org
Read 19210 from http://www.jython.org
Read 19210 from http://www.jython.org
Read 19210 from http://www.jython.org
Read 19210 from http://www.jython.org
Read 19210 from http://www.jython.org
Read 19210 from http://www.jython.org
Read 19210 from http://www.jython.org
Read 19210 from http://www.jython.org
Read 19210 from http://www.jython.org
Read 19210 from http://www.jython.org
Read 19210 from http://www.jython.org
Read 19210 from http://www.jython.org
Read 19210 from http://www.jython.org
Read 19210 from http://www.jython.org
Read 19210 from http://www.jython.org
Read 19210 from http://www.jython.org
Read 19210 from http://www.jython.org
Read 19210 from http://www.jython.org
Read 19210 from http://www.jython.org
Read 19210 from http://www.jython.org
Read 19210 from http://www.jython.org
Read 19210 from http://www.jython.org
Read 19210 f

### `multiprocessing` Version ###


In [ ]:
import requests
import multiprocessing
import time

session = None

def set_global_session():
    global session
    if not session:
        session = requests.Session()
        
def download_site(url):
    with session.get(url) as response:
        name = multiprocessing.current_process().name
        print(f"{name}: Read {len(response.content)} from {url}")
        
def download_all_sites(sites):
    with multiprocessing.Pool(initializer=set_global_session) as pool:
        pool.map(download_site, sites)
        
if __name__ == "__main__":
    sites = [
        "http://www.jython.org",
        "http://olympus.realpython.org/dice",
    ] * 80

    start_time = time.time()
    download_all_sites(sites)
    duration = time.time() - start_time
    print(f"Downloaded {len(sites)} in {duration} seconds")        

### CPU-Bound  Synchronous Version###

In [1]:
import time

def cpu_bound(number):
    return sum(i*i for i in range(number))

def find_sums(numbers):
    for number in numbers:
        cpu_bound(number)
        
if __name__ == '__main__':
    numbers = [5_00_000 + x for x in range(20)]
    
    start_time = time.time()
    find_sums(numbers)
    duration = time.time() - start_time
    print(f"Duration {duration} seconds")        
        

Duration 5.865610361099243 seconds


### CPU-Bound `multiprocessing` Version ###

In [ ]:
import multiprocessing
import time

def cpu_bound(number):
    return sum(i*i for i in range(number))

def find_sums(numbers):
    with multiprocessing.Pool() as pool:
        pool.map(cpu_bound, numbers)
        
if __name__ == '__main__':
    numbers = [5_00_000 + x for x in range(20)]
    
    start_time = time.time()
    find_sums(numbers)
    duration = time.time() - start_time
    print(f"Duration {duration} seconds")        
        